# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: *** @Rebecamoreno_***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = 'Nutella'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [7]:
dft = pd.read_excel('./{0}.xlsx'.format(produto),sheet='Treinamento')

dftsim = dft[(dft.Relevante=="Sim")]
dftnao = dft[(dft.Relevante=="Não")]

In [8]:
lendo = pd.ExcelFile('./{0}.xlsx'.format(produto))
dfteste = lendo.parse("Teste")

In [22]:
# A, S e N são dicionários com as frequências de cada uma das palavras separadas (splitadas).
A = {}
S = {}
N = {}
todas = 0
c=0
#para cada post (msg)
for msg in dft["Treinamento"]:
    #para cada palavra dentro do post (i)
    #msg = msg.split()
    for i in msg.split():
        #analisando cada palavra ja separada (for e if)
        for caracter in range(0,len(i)):
            i = i.strip(",").strip("'").strip('"').strip('#').strip(':').strip(';').strip('!').strip('(').strip(')').strip('/').strip('\n').strip('.').strip('\\').strip('-').strip('$').strip('%').strip('|').strip('=').strip('*').strip('ˆ').strip('&').strip('+').strip('?')
        #contando sua frenquencia e adicionando em A
        if not i in A:
            A[i]=0
        A[i]+=1
        todas+=1
        if A[i]>=c:
            c=A[i]
            string = i

print('Como curiosidade, a palavra que aparece com mais frequência em toda essa análise é a palavra "{0}", com uma frenquência de {1} vezes.\n'.format(string,c))
todas_em_sim = 0
p = 0
#para cada post (msg)
for msg in dftsim["Treinamento"]:
    #para cada palavra dentro do post (i)
    #msg = msg.split()
    for i in msg.split():
        #analisando cada palavra ja separada (for e if)
        for caracter in range(0,len(i)):
            i = i.strip(",").strip("'").strip('"').strip('#').strip(':').strip(';').strip('!').strip('(').strip(')').strip('/').strip('\n').strip('.').strip('\\').strip('-').strip('$').strip('%').strip('|').strip('=').strip('*').strip('ˆ').strip('&').strip('+').strip('?')
        #contando sua frenquencia e adicionando em A
        if not i in S:
            S[i]=0
        S[i]+=1
        todas_em_sim+=1
        if S[i]>=p:
            p=S[i]
            strings = i
            
print('Como curiosidade, a palavra que aparece com mais frequência entre todas as palavras de RELEVANTES é a palavra "{0}", com uma frenquência de {1} vezes.\n'.format(strings,p))
todas_em_nao = 0
f = 0
#para cada post (msg)
for msg in dftnao["Treinamento"]:
    #para cada palavra dentro do post (i)
    #msg = msg.split()
    for i in msg.split():
        #analisando cada palavra ja separada (for e if)
        for caracter in range(0,len(i)):
            i = i.strip(",").strip("'").strip('"').strip('#').strip(':').strip(';').strip('!').strip('(').strip(')').strip('/').strip('\n').strip('.').strip('\\').strip('-').strip('$').strip('%').strip('|').strip('=').strip('*').strip('ˆ').strip('&').strip('+').strip('?')
        #contando sua frenquencia e adicionando em A
        if not i in N:
            N[i]=0
        N[i]+=1
        todas_em_nao+=1
        if N[i]>=f:
            f=N[i]
            stringn = i
            
print('Como curiosidade, a palavra que aparece com mais frequência entre todas as palavras de NÃO RELEVANTES é a palavra "{0}", com uma frenquência de {1} vezes.\n'.format(stringn,f))
print('A quantidade TOTAL de palavras no UNIVERSO é {0}.'.format(todas))
print('A quantidade TOTAL de palavras dentro de RELEVANTES é {0}.'.format(todas_em_sim))
print('A quantidade TOTAL de palavras dentro de NÃO RELEVANTES é {0}.\n'.format(todas_em_nao))
print('A quantidade de palavras DIFERENTES no UNIVERSO é {0}.'.format(len(A)))
print('A quantidade de palavras DIFERENTES dentro de RELEVANTES é {0}.'.format(len(S)))
print('A quantidade de palavras DIFERENTES dentro de NÃO RELEVANTES é {0}.'.format(len(N)))

Como curiosidade, a palavra que aparece com mais frequência em toda essa análise é a palavra "nutella", com uma frenquência de 219 vezes.

Como curiosidade, a palavra que aparece com mais frequência entre todas as palavras de RELEVANTES é a palavra "nutella", com uma frenquência de 135 vezes.

Como curiosidade, a palavra que aparece com mais frequência entre todas as palavras de NÃO RELEVANTES é a palavra "nutella", com uma frenquência de 84 vezes.

A quantidade TOTAL de palavras no UNIVERSO é 3774.
A quantidade TOTAL de palavras dentro de RELEVANTES é 1750.
A quantidade TOTAL de palavras dentro de NÃO RELEVANTES é 2024.

A quantidade de palavras DIFERENTES no UNIVERSO é 1370.
A quantidade de palavras DIFERENTES dentro de RELEVANTES é 636.
A quantidade de palavras DIFERENTES dentro de NÃO RELEVANTES é 917.


In [112]:
#Calculando a probabilidade de cada palavra
#Conta: P(sim|palavra)=(P(palavra|sim)+1) / P(palavras totais|sim)+P(palavras totais sem repetição)
bs = todas_em_sim
bn = todas_em_nao
c = len(A)
PS = {}
PN = {}

#PTeste = {}

for msg2 in dfteste['Teste']:
    #Para sim:
    Ptweet = 1
    for i in msg2.split():
        #analisando cada palavra ja separada (for e if)
        for caracter in range(0,len(i)):
            i = i.strip(",").strip("'").strip('"').strip('#').strip(':').strip(';').strip('!').strip('(').strip(')').strip('/').strip('\n').strip('.').strip('\\').strip('-').strip('$').strip('%').strip('|').strip('=').strip('*').strip('ˆ').strip('&').strip('+').strip('?')
        #contando sua frenquencia e adicionando em A
        
        if i in S:
            PS[i] = (S[i]+1)/(bs+c)
            
        else:
            PS[i] = 1/(bs+c)
            
        Ptweet = Ptweet*PS[i]
        print(Ptweet)
        
    #Para não:   
    for i in msg2.split():
        #analisando cada palavra ja separada (for e if)
        for caracter in range(0,len(i)):
            i = i.strip(",").strip("'").strip('"').strip('#').strip(':').strip(';').strip('!').strip('(').strip(')').strip('/').strip('\n').strip('.').strip('\\').strip('-').strip('$').strip('%').strip('|').strip('=').strip('*').strip('ˆ').strip('&').strip('+').strip('?')
        #contando sua frenquencia e adicionando em A
        
        if i in N:
            PN[i] = (N[i]+1)/(bn+c)
            
        else:
            PN[i] = 1/(bn+c)
        
#print(PN)
#print(PS["nutella"])

0.011538461538461539
3.6982248520710063e-06
1.1853284782278866e-09
1.06375632661477e-11
6.818950811633141e-15
8.742244630298898e-18
5.60400296814032e-21
3.592309594961743e-24
6.908287682618738e-27
2.2141947700701084e-30
1.277420059655832e-32
4.094295062999461e-36
1.1679239122017693e-37
3.743345872441568e-41
1.1997903437312718e-44
0.0003205128205128205
2.8763971071663377e-06
9.219221497328005e-10
4.727805896065643e-12
2.5760480843947413e-14
8.256564373060068e-18
2.64633473495515e-21
8.481842099215225e-25
2.41949982958383e-26
7.75480714610202e-30
2.4855151109301346e-33
5.5764762104201734e-36
1.0723992712346488e-38
1.0655249169318625e-40
1.810026301198356e-42
7.889858235992834e-44
2.5288007166643695e-47
8.10513050212939e-51
2.5977982378619837e-54
0.0003205128205128205
1.027284681130835e-07
3.292579106188574e-11
1.0553138160860814e-14
3.382416077198979e-18
1.0841077170509547e-21
1.5983639418058947e-23
5.12296135194197e-27
1.641974792289093e-30
4.6838383497990147e-32
2.041673126835468e-33
0

2.4334908460079378e-30
2.417891545713015e-32
1.5499304780211636e-35
0.0009615384615384616
2.7736686390532547e-06
2.133591260810196e-08
6.838433528237808e-12
1.0959028090124692e-14
2.1075054019470564e-17
2.0939957519345751e-19
3.0873014291343094e-21
1.3457467768021348e-22
4.3132909512888935e-26
0.007692307692307693
3.451676528599605e-05
9.956759217114246e-08
8.935553143564066e-10
2.863959340885919e-13
8.16962760701432e-15
8.902799315336117e-17
8.845730088955757e-19
1.502640047162356e-20
6.549969436348731e-22
2.099349178316901e-25
6.728683263836221e-28
4.313258502459115e-31
2.764909296448151e-34
0.0003205128205128205
2.05456936226167e-07
1.119476896104115e-09
4.8797710855820396e-11
0.0003205128205128205
1.027284681130835e-07
6.585158212377148e-11
2.1106276321721628e-14
1.0823731447036733e-16
3.469144694563055e-20
1.111905350821492e-23
3.563799201350936e-27
1.553450933922203e-28
4.979009403596804e-32
2.234170886229335e-34
7.16080412252992e-38
6.885388579355692e-41
4.4137106277921106e-44
1

1.1539010353288146e-47
3.698400754259021e-51
1.1853848571343017e-54
3.799310439533018e-58
1.9483643279656503e-60
3.122378730714183e-63
1.5011436205356649e-65
4.811357758127131e-69
3.084203691107135e-72
0.007692307692307693
7.396449704142013e-06
3.793051130329237e-08
1.2157215161311658e-11
1.1689629962799671e-14
5.095479727374215e-16
5.552766369574465e-18
1.7797328107610467e-21
5.704271829362329e-25
0.007692307692307693
7.396449704142013e-06
4.0301168259748144e-08
2.5834082217787272e-11
2.48404636709493e-14
7.961687074022211e-18
8.676197452460102e-20
1.3904162584070676e-22
3.119523656682523e-25
1.699740453961631e-27
5.4478860703898425e-31
1.7461173302531545e-34
1.9028201675835658e-36
6.098782588408865e-40
1.7397168281038106e-41
2.7880077373458505e-44
4.110524228138113e-46
6.587378570734155e-49
4.2226785709834326e-52
1.8947916664669247e-54
8.50227029824902e-57
5.450173268108346e-60
9.258307154158409e-62
4.035672349248537e-63
0.011538461538461539
3.6982248520710063e-06
1.1853284782278866e

In [105]:
for msg2 in dfteste['Teste']:
    PTweet = 1
    for i in msg2.split():
        print(PS[i])
        #PTweet = PTweet*P[i]
    '''
        for i in msg2.split():
            for caracter in range(0,len(i)):
                i = i.strip(",").strip("'").strip('"').strip('#').strip(':').strip(';').strip('!').strip('(').strip(')').strip('/').strip('\n').strip('.').strip('\\').strip('-').strip('$').strip('%').strip('|').strip('=').strip('*').strip('ˆ').strip('&').strip('+').strip('?')
            #print(i)'''

        
        

0.011538461538461539


KeyError: '@br00talgirl:'

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
